# Homework 1

## Task 1 - Wordle

*To clone the wordle puzzle game, the "puzzle_solver" function is defined to return the results of guessing within six rounds. To compare the guess results and prevent unintentional modification, a new word list is created at the beginning by copying the original wordle text. The correctness and index information are detected by the color of returning puzzle object. If it is green, the words containing that appropriate guess will be picked out； If yellow, it will be the words with the dismatched alphabet；The rest will be the grey one, after checking the possible duplication cases by sum of letters, it will be filtered out from the words list. Then, we will give a point to each word by checking whether it has the most frequent character in each column, and finally the optimal solution will be returned*

In [1]:
import pandas as pd
import numpy as np
import wordle

In [2]:
# create a dataframe for wordle word list
df = pd.read_csv("data/wordle.txt", sep=" ", header=None, names=["word"])
df[["rm1","char1","char2","char3","char4","char5","rm2"]] = df["word"].str.split('',expand=True)
df.drop(['word','rm1', 'rm2'], axis=1, inplace=True)

def puzzle_solver(p):
    """
    Solve the Wordle puzzle

    -------
    p:
        the results of guessing
    """
    word_list = df.copy()
    word = "crane"            # start with "crane"
    for i in range(0,6):
        res = p.guess(word)
        if p.is_solved()==True:
            return p
        green_index = []
        green_char = []
        yellow_index = []
        grey_index = []
        yellow_char = []
        grey_char = []
        #color detection
        for i in range(0,5):       #for each character
            if res[i]==u"\U0001F7E9":         # "U0001F7E9": green
                green_char.append(word[i])
                green_index.append(i)
            elif res[i] == u"\U0001F7E8":     # "U0001F7E8": yellow
                yellow_char.append(word[i])
                yellow_index.append(i)
            else:
                grey_char.append(word[i])
                grey_index.append(i)

        # filter by green        
        for green in range(0,len(green_index)):
            g_index = green_index[green]
            g_char = green_char[green]
            word_list = word_list[word_list.iloc[:,g_index]==g_char] 
           
        # filter by yellow   
        for y in range(0,len(yellow_char)):
            y_index = yellow_index[y]    
            y_char = yellow_char[y]
            word_list = word_list[word_list.iloc[:,y_index]!=y_char]   # with the correct letter but wrong position
            # check how many yellow letters in each word
            sum_row = word_list.iloc[:,yellow_index+grey_index].apply(lambda row: row==y_char).sum(axis=1)   
            occur = yellow_char.count(y_char)
            word_list = word_list[list(sum_row>=occur)]

        # filter by grey
        for g in range(0,len(grey_char)):
            g_char = grey_char[g]
            sum_row = word_list.iloc[:,yellow_index+grey_index].apply(lambda row: row==g_char).sum(axis=1)
            # check the duplication case for grey and yellow 
            if g_char not in yellow_char: 
                word_list = word_list[list(sum_row==0)]
            else:
                y_count = yellow_char.count(g_char)   
                word_list = word_list[list(sum_row==y_count)]
                
        mode_char = list(word_list.mode().iloc[0])
        total_point = [0]*len(word_list.index)
        
        # find out the optimal solution by most frequent char in each column
        for i in range(0,len(mode_char)):
            point = list(word_list.iloc[:,i]==mode_char[i])
            total_point = [x+y for x,y in zip(point,total_point)]
        rec_index = total_point.index(max(total_point))
        word = ''.join(word_list.iloc[rec_index])
    return p

### Assessment

In [3]:
import random
random.seed(1234)
n = 200

# Change both the seed value and `n` of puzzles to get 
# a more accurate view of your solver's performance.

puzzles  = [wordle.puzzle() for i in range(n)]
attempts = [puzzle_solver(p) for p in puzzles]
solved   = [p for p in puzzles if p.is_solved()]
n_guess  = [p.n_guesses() for p in solved]

print(f"Solved {len(solved)} of {len(puzzles)} puzzles attempted.")
if (len(solved) != 0):
    print(f"Average # of guesses required: {sum(n_guess)/len(n_guess)}")                         

Solved 183 of 200 puzzles attempted.
Average # of guesses required: 4.420765027322404


## Task 2

*Advent_step is used for updating the next step of the matrix, and advent_solve is used for counting the steps to reach the steady step. Due to the python list nature, we use deep copy in this case.<br><br>Advent_step: Since the matrix moving contains special case, we first check the special case. The special case is that ‘>’ is the last element of the last column and ‘.’ is the first element in the same row. In this case, we make the first element ‘>’ and the current element ‘.’. In the normal case, we check whether the right next element is ‘.’. If the right next element is ‘.’, which means we can move the matrix, so the right next element become ‘>’ and current element become ‘.’. If the right next element is ‘>’, then we cannot move ‘>’. The same logic follows for ‘v’ case.<br><br>Advent_solve: We are counting the total step to reach the steady state. We check by using current_step != next_step, if not then it means the matrix is moving and we add counts. If current_step == next_step, then it means the matrix cannot move anymore and reach steady state.*

In [4]:
import advent
from copy import deepcopy

In [5]:
def advent_step(state):
    """
    Include a docstring
    
    This function that iterates this system(a matrix which is composed of the character values:'.','>' and'v') 
    until it reaches final steady state
    
    state(list):a matrix which is composed of the character values: “.”, “>”, and “v”
    
    next_step(list): a copy matrix to perform right moves and down moves

    """

    # make a copy at first to avoid overwrite the original state
    # deepcopy
    next_step = deepcopy(state)

    # loop matrix row
    for i in range(len(state)):
        # loop matrix column
        for j in range(len(state[0])):
            if state[i][j] == '>':
                # special case: check if '>' is the last element
                if j == len(state[0]) - 1:
                    # special case: check if '.' is the first element
                    if state[i][0] == '.':
                        next_step[i][0] = '>'
                        next_step[i][j] = '.'
                # normal case
                elif state[i][j+1] == '.':
                    next_step[i][j+1] = '>'
                    next_step[i][j] = '.'
    
    state = deepcopy(next_step)

    # loop matrix row
    for i in range(len(state)):
        # loop matrix column
        for j in range(len(state[0])):
            if state[i][j] == 'v':
                # special case: check if 'v' is the last element
                if i == len(state) - 1:
                    # special case: check if '.' is the first element
                    if state[0][j] == '.':
                        next_step[0][j] = 'v'
                        next_step[i][j] = '.'
                # normal case
                elif state[i+1][j] == '.':
                    next_step[i+1][j] = 'v'
                    next_step[i][j] = '.'

    return next_step

In [6]:
def advent_solve(state):
    """
    Include a docstring
    
    state (list): a matrix which is composed of the character values: “.”, “>”, and “v”
    
    count (int): the steps used to evolve state into a steady matrix
    """
    # initial step
    current_step = deepcopy(state)
    next_step = advent_step(current_step)
    count = 1

    # keep running till converge
    while next_step != current_step:
        current_step = deepcopy(next_step)
        next_step = advent_step(current_step)
        count = count + 1
    
    return count

In [7]:
# Check that the implementation above returns the correct results
assert advent_solve(advent.small) == 58
assert advent_solve(advent.large) == 351

## Task 3

*In this question, we also have two functions, advent_step_np follows the same logic as advent_step in task 2.<br>Advent_solve_np follows the same logic as advent_solve in task 2. Since we are using np in this task, the simple copy is sufficient.*

In [8]:
import advent
import numpy as np

In [9]:
def advent_step_np(state):
    """
    Include a docstring
    This function that iterates this system(a matrix which is composed of the character values:'.','>' and'v') 
    until it reaches final steady state
    
    state(list):a matrix which is composed of the character values: “.”, “>”, and “v”
    
    next_step(list): a copy matrix to perform right moves and down moves
    """
    # make a copy at first to avoid overwrite the original state
    # copy 
    next_step = state.copy()
    
    # loop over row
    for i in range(len(state)):
        # loop over column
        for j in range(len(state[0])):
            if state[i][j] == '>':
                # special case: check if '>' is the last element
                if j == len(state[0]) - 1:
                    # special case: check if '.' is the first element
                    if state[i][0] == '.':
                        next_step[i][0] = '>'
                        next_step[i][j] = '.'
                # normal case
                elif state[i][j+1] == '.':
                    next_step[i][j+1] = '>'
                    next_step[i][j] = '.'
    
    state = next_step.copy()

    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] == 'v':
                # special case check if 'v' is the last element and '.' is the first element
                if i == len(state) - 1:
                    if state[0][j] == '.':
                        next_step[0][j] = 'v'
                        next_step[i][j] = '.'
                # normal case
                elif state[i+1][j] == '.':
                    next_step[i+1][j] = 'v'
                    next_step[i][j] = '.'

    return next_step

In [10]:
def advent_solve_np(state):
    """
    Include a docstring
    
    state (list): a matrix which is composed of the character values: “.”, “>”, and “v”
    
    count (int): the steps used to evolve state into a steady matrix
    """
    # initial step
    current_step = state.copy()
    next_step = advent_step(current_step)
    count = 1

    # keep running until converge
    while not np.array_equal(current_step, next_step):
        current_step = next_step.copy()
        next_step = advent_step(current_step)
        count = count + 1
    
    return count

In [11]:
# Check that the implementation above returns the correct results
assert advent_solve_np(advent.small_np) == 58
assert advent_solve_np(advent.large_np) == 351

### Performance

In [12]:
%timeit -r 3 advent_solve(advent.large)

8.96 s ± 66.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [13]:
%timeit -r 3 advent_solve_np(advent.large)

8.07 s ± 12.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


*Discuss: which implementation was faster? Explain why you think this is?*

*In our implementation, the numpy version is around 1 second faster.<br>1. In the normal version, we use deep_copy to copy python list of list list but in numpy version we only use copy. Copy is faster and memory saving method in python.<br>2. Data in Numpy nd-array has homogeneous data type while python list can contain heterogeneous data type. It is possible that in the default setting, homogeneous data type computation is faster than heterogeneous data type.*